#  Машинный перевод с использованием рекуррентных нейронных сетей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#the-seq2seq-model
* https://www.adeveloperdiary.com/data-science/deep-learning/nlp/machine-translation-recurrent-neural-network-pytorch/

## Задачи для совместного разбора

1\. Рассмотрите пример архитектуры Encoder-Decoder с использованием RNN. Обсудите концепцию teacher forcing.

In [1]:
import torch as th
import torch.nn as nn

In [21]:
n_ru_tokens = 1000
batch_size = 16
ru_seq_len = 15
n_en_tokens = 500
en_seq_len = 10

ru = th.randint(0, n_ru_tokens, size=(batch_size, ru_seq_len))
en = th.randint(0, n_en_tokens, size=(batch_size, en_seq_len))

In [22]:
class Encoder(nn.Module):
  def __init__(self, embedding_dim, hidden_size):
      super().__init__()
      self.emb = nn.Embedding(
          num_embeddings=n_ru_tokens,
          embedding_dim=embedding_dim,
          padding_idx=0
      )
      self.dropout = nn.Dropout(p=0.5)
      self.rnn = nn.GRU(embedding_dim, hidden_size, batch_first=True)

  def forward(self, X):
    out = self.emb(X) # batch x seq x emb_size
    out = self.dropout(out)
    _, h = self.rnn(out) # out: batch x seq x hidden_size
    return h # 1 x batch x hidden_size

In [23]:
embedding_dim = 100
encoder_hidden_size = 300

encoder = Encoder(embedding_dim, encoder_hidden_size)

encoder_output = encoder(ru)

In [24]:
encoder_output.shape

torch.Size([1, 16, 300])

In [27]:
class Decoder(nn.Module):
  def __init__(self, embedding_dim, decoder_hidden_size):
    super().__init__()
    self.emb = nn.Embedding(
          num_embeddings=n_en_tokens,
          embedding_dim=embedding_dim,
          padding_idx=0
    )
    self.rnn = nn.GRUCell(embedding_dim, decoder_hidden_size)
    self.fc = nn.Linear(decoder_hidden_size, n_en_tokens)

  def forward(self, encoder_output, labels):
    # labels: batch x seq_len - считаем, что в 0 столбце SOS
    # encoder_output: 1 x batch x encoder_hidden_size
    seq_len = labels.size(1)
    input_tokens = labels[:, 0]
    decoder_hidden = encoder_output[0]
    for _ in range(1, seq_len):
      out = self.emb(input_tokens).relu() # batch x emb_size
      decoder_hidden = self.rnn(out, decoder_hidden) # batch x dec_hidden
      out = self.fc(decoder_hidden) # batch x n_en_tokens

      # teacher forcing
      input_tokens = out.argmax(dim=1).detach()
      # ...


    # вернуть прогнозы для каждого эл-та последовательности
    # batch x seq x n_en_token
    return ...

In [29]:
decoder = Decoder(embedding_dim=100, decoder_hidden_size=300)
decoder(encoder_output, en)

Ellipsis

## Задачи для самостоятельного решения

In [2]:
import torchmetrics

In [3]:
import pandas as pd
import numpy as np
import torch
import torchtext
from torch.utils.data import TensorDataset, Dataset, DataLoader
import torchtext.transforms as tr
import torch.nn as nn

<p class="task" id="1"></p>

1\. Считайте файлы `RuBQ_2.0_train.json` (обучающее множество) и `RuBQ_2.0_test.json` (тестовое множество). Для каждого файла создайте по списка: список предложений на русском языке и список предложений на английском языке. Выведите на экран количество примеров в обучающей и тестовой выборке.

- [ ] Проверено на семинаре

In [4]:
df_train = pd.read_json('data/RuBQ_2.0_train.json')
df_test = pd.read_json('data/RuBQ_2.0_test.json')
df_test = df_test.loc[:, ['question_text', 'question_eng']]
df_train = df_train.loc[:, ['question_text', 'question_eng']]
ru_train = df_train['question_text'].values
eng_train = df_train['question_eng'].values
ru_test = df_test['question_text'].values
eng_test = df_test['question_eng'].values

In [5]:
ru_test.shape, ru_train.shape

((580,), (2330,))

<p class="task" id="2"></p>

2\. Создайте два Vocab на основе загруженных данных: `ru_vocab` для слов на русском языке и `en_vocab` для слов на английском языке (словари создаются на основе обучающего множества). Добавьте в словари специальные токены `<PAD>`, `<SOS>`, `<EOS>`. Выведите на экран количество токенов в полученных словарях. Выведите на экран максимальное кол-во слов в предложениях на русском языке и в предложениях на английском языке (в обучающей выборке).

- [ ] Проверено на семинаре

In [6]:
import re
regex = re.compile(r"[^а-яА-Яa-zA-Z\s]")

In [7]:
ru_train = list(map(lambda x: regex.sub('', x).lower().split(), ru_train.tolist()))
eng_train = list(map(lambda x: regex.sub('', x).lower().split(), eng_train.tolist()))

In [8]:
vocab_ru = torchtext.vocab.build_vocab_from_iterator(ru_train, specials=['<unk>', '<pad>', '<sos>', '<eos>'], special_first=[0, 1, 2, 3])
vocab_eng = torchtext.vocab.build_vocab_from_iterator(eng_train, specials=['<unk>', '<pad>', '<sos>', '<eos>'], special_first=[0, 1, 2, 3])
vocab_ru.set_default_index(0)
vocab_eng.set_default_index(0)

In [9]:
len(vocab_ru), len(vocab_eng)

(5825, 4234)

In [10]:
print(f'Максимум на русском {pd.Series(ru_train).apply(len).max()}')
print(f'Максимум на eng {pd.Series(eng_train).apply(len).max()}')
max_eng = pd.Series(eng_train).apply(len).max()
max_rus = pd.Series(ru_train).apply(len).max()

Максимум на русском 25
Максимум на eng 31


<p class="task" id="3"></p>

3\. Создайте класс `RuEnDataset`. Реализуйте `__getitem__` таким образом, чтобы он возвращал кортеж `(x, y)`, где x - это набор индексов токенов для предложений на русском языке, а `y` - набор индексов токенов для предложений на английском языке. Используя преобразования, сделайте длины наборов индексов одинаковой фиксированной длины, добавьте в начало каждого набора индекс `<SOS>`, а в конец - индекс токена `<EOS>`. Создайте датасет для обучающей и тестовой выборки.

- [ ] Проверено на семинаре

In [11]:
X_transform = tr.Sequential(
    tr.AddToken(token="<sos>", begin=True),
    tr.AddToken(token="<eos>", begin=False),
    tr.VocabTransform(vocab_ru),
    tr.Truncate(max_seq_len=max_rus),
    tr.ToTensor(),
    tr.PadTransform(max_length=max_rus, pad_value=1))
y_transform = tr.Sequential(
    tr.AddToken(token="<sos>", begin=True),
    tr.AddToken(token="<eos>", begin=False),
    tr.VocabTransform(vocab_eng),
    tr.Truncate(max_seq_len=max_eng),
    tr.ToTensor(),
    tr.PadTransform(max_length=max_eng, pad_value=1)
)

In [12]:
class RuEnDataset(Dataset):
    def __init__(self, X, y, X_transform, y_transform):
        super(RuEnDataset, self).__init__()
        self.X_data = X
        self.y_data = y
        self.X_transform = X_transform
        self.y_transform = y_transform
    def __getitem__(self, idx):
        current_X = self.X_transform(self.X_data[idx])
        current_y = self.y_transform(self.y_data[idx])
        return current_X, current_y

    def __len__(self):
        return len(self.X_data)

In [13]:
ru_test = list(map(lambda x: regex.sub('', x).lower().split(), ru_test.tolist()))
eng_test = list(map(lambda x: regex.sub('', x).lower().split(), eng_test.tolist()))

In [14]:
train_dataset = RuEnDataset(X=ru_train, y=eng_train, X_transform=X_transform, y_transform=y_transform)
test_dataset = RuEnDataset(X=ru_test, y=eng_test, X_transform=X_transform, y_transform=y_transform)

In [15]:
train_dataset[0]

(tensor([   2,   27,  677, 2206, 5594,    3,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1]),
 tensor([   2,    6,  148,  235,   22, 3980,    3,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1]))

In [16]:
test_dataset[1]

(tensor([   2,   29,    5,  681, 2399,    0, 4758,   90,    0,    0,    3,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1]),
 tensor([  2,  10, 224, 142,   7,   0,   0,  16,   0,   0,  15,   3,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1]))

<p class="task" id="4"></p>

4\. Опишите модель `Encoder`, которая возвращает скрытое состояние рекуррентного слоя в соотстветствии со следующей схемой. Пропустите через эту модель первые 16 предложений на русском языке и выведите размер полученного результата на экран. Результатом должен являться тензор размера `1 x batch_size x hidden_dim` (если используется один однонаправленный рекуррентный слой и `batch_first=True`).

* количество эмбеддингов равно количеству слов на русском языке;
* размерность эмбеддингов выберите самостоятельно;
* при создании слоя эмбеддингов укажите `padding_idx`;
* размер скрытого состояния рекуррентного слоя выберите самостоятельно.

![encoder](https://i0.wp.com/www.adeveloperdiary.com/wp-content/uploads/2020/10/Machine-Translation-using-Recurrent-Neural-Network-and-PyTorch-adeveloperdiary.com-1.png?w=815&ssl=1)

- [ ] Проверено на семинаре

In [17]:
class Encoder(nn.Module):
  def __init__(self, embedding_dim, hidden_size, n_ru_tokens):
      super().__init__()
      self.emb = nn.Embedding(
          num_embeddings=n_ru_tokens,
          embedding_dim=embedding_dim,
          padding_idx=0
      )
      self.dropout = nn.Dropout(p=0.5)
      self.rnn = nn.GRU(embedding_dim, hidden_size, batch_first=True)

  def forward(self, X):
    out = self.emb(X) # batch x seq x emb_size
    out = self.dropout(out)
    _, h = self.rnn(out) # out: batch x seq x hidden_size
    return h # 1 x batch x hidden_size

In [18]:
encoder = Encoder(embedding_dim=30, n_ru_tokens=len(vocab_ru), hidden_size=20)

In [19]:
dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)
for X, y in dataloader:
    print(encoder(X).shape)
    break

torch.Size([1, 16, 20])


<p class="task" id="5"></p>

5\. Опишите модель `Decoder`, которая возвращает прогноз (набор индексов слов на английском языке). Пропустите через эту модель тензор скрытых состояний кодировщика, полученный в предыдущей задачи, и выведите размер полученного результата на экран. Результатом должен являться тензор размера `batch_size x seq_len x n_en_words` (если используется один однонаправленный рекуррентный слой и `batch_first=True`).

* количество эмбеддингов равно количеству слов на английском языке;
* размер выходного слоя равен количеству слов на английском языке;
* размерность эмбеддингов выберите самостоятельно;
* при создании слоя эмбеддингов укажите `padding_idx`;
* размер скрытого состояния рекуррентного слоя выберите самостоятельно.

![decoder](https://i2.wp.com/www.adeveloperdiary.com/wp-content/uploads/2020/10/Machine-Translation-using-Recurrent-Neural-Network-and-PyTorch-adeveloperdiary.com-2.png?w=899&ssl=1)

- [ ] Проверено на семинаре

In [20]:
class Decoder(nn.Module):
  def __init__(self, embedding_dim, decoder_hidden_size, n_en_tokens, epsilon=0.5):
    super().__init__()
    self.emb = nn.Embedding(
          num_embeddings=n_en_tokens,
          embedding_dim=embedding_dim,
          padding_idx=1
    )
    self.rnn = nn.GRUCell(embedding_dim, decoder_hidden_size)
    self.fc = nn.Linear(decoder_hidden_size, n_en_tokens)
    self.epsilon = epsilon


  def forward(self, encoder_output, labels):
    # labels: batch x seq_len - считаем, что в 0 столбце SOS
    # encoder_output: 1 x batch x encoder_hidden_size
    seq_len = labels.size(1)
    to_ret = []
    input_tokens = labels[:, 0]
    decoder_hidden = encoder_output[0]
    for _ in range(1, seq_len):
      out = self.emb(input_tokens).relu() # batch x emb_size
      decoder_hidden = self.rnn(out, decoder_hidden) # batch x dec_hidden
      out = self.fc(decoder_hidden) # batch x n_en_tokens
      to_ret.append(out)
      # teacher forcing
      if torch.rand(1).item() > self.epsilon:
          input_tokens = out.argmax(dim=1).detach()
      else:
          input_tokens = labels[:, _]


    # вернуть прогнозы для каждого эл-та последовательности
    # batch x seq x n_en_token
    return torch.stack(to_ret).permute(1, 0, 2)

In [21]:
decoder = Decoder(embedding_dim=30, decoder_hidden_size=20, n_en_tokens=len(vocab_eng))
encoder = Encoder(embedding_dim=30, n_ru_tokens=len(vocab_ru), hidden_size=20)

dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)
for X, y in dataloader:
    print(decoder(encoder(X), y).shape)
    break

torch.Size([16, 30, 4234])


<p class="task" id="6"></p>

6\. Объедините модели `Encoder` и `Decoder` в одну модель `EncoderDecoder`. Пропустите через эту модель первые 16 предложений на русском языке и выведите размер полученного результата на экран. Сделайте полученный результат двумерным, объединив две первые размерности: `batch_size * seq_len x n_en_words`. Выведите размерность полученного результата на экран.

- [ ] Проверено на семинаре

In [22]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, n_en_tokens):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.n_en_tokens = n_en_tokens


    def forward(self, X, labels):
        encoder_output = self.encoder(X)
        out = self.decoder(encoder_output, labels)
        out = out.reshape(-1, self.n_en_tokens)
        return out

In [23]:
decoder = Decoder(30, 20, len(vocab_eng))
encoder = Encoder(embedding_dim=30, n_ru_tokens=len(vocab_ru), hidden_size=20)
encoder_decoder = EncoderDecoder(encoder, decoder, len(vocab_eng))
dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)
for X, y in dataloader:
    print(encoder_decoder(X, y).shape)
    break

torch.Size([480, 4234])


<p class="task" id="7"></p>

7\. Настройте модель, решив задачу классификации на основе прогнозов модели `EncoderDecoder`. Игнорируйте токен `<PAD>` при расчете ошибки. Во время обучения выводите на экран значения функции потерь для эпохи (на обучающем множестве), значение accuracy по токенам (на обучающем множестве) и пример перевода, сгенерированного моделью. После завершения обучения посчитайте BLEU для тестового множества.

- [ ] Проверено на семинаре

In [24]:
from tqdm import tqdm

In [45]:
epochs = 500
batch_size = 128
device = torch.device('cuda')
decoder = Decoder(embedding_dim=150, decoder_hidden_size=100, n_en_tokens=len(vocab_eng))
encoder = Encoder(embedding_dim=150, n_ru_tokens=len(vocab_ru), hidden_size=100)
model = EncoderDecoder(encoder, decoder, len(vocab_eng)).to(device)

train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=True)
loss = nn.CrossEntropyLoss(ignore_index=1)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

specials= set(['<unk>', '<pad>', '<sos>', '<eos>'])


In [47]:
for i in range(epochs):
    train_accuracy = torchmetrics.Accuracy(num_classes=len(vocab_eng), task='multiclass').to(device)
    test_accuracy = torchmetrics.Accuracy(num_classes=len(vocab_ru), task='multiclass').to(device)

    model.train()
    train_loss = 0
    test_loss = 0
    for current_X, current_y in train_dataloader:
        current_X = current_X.to(device)
        current_y = current_y.to(device)
        pred = model(current_X, current_y)
        current_y = current_y[:, 1:] # так как предикты без <sos>
        error = loss(pred, current_y.flatten().long())
        arg_pred = torch.argmax(pred, dim = 1)
        train_accuracy.update(arg_pred, current_y.flatten().long())
        error.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += error.item()

    model.eval()
    for current_X, current_y in test_dataloader:
        current_X = current_X.to(device)
        current_y = current_y.to(device)
        pred = model(current_X, current_y)
        current_y = current_y[:, 1:]
        error = loss(pred, current_y.flatten().long())
        arg_pred = torch.argmax(pred, dim = 1)
        test_accuracy.update(arg_pred, current_y.flatten().long())
        test_loss += error.item()
    if not i%25:
        model.eval()
        index = torch.randint(0, len(train_dataset), size=(1,))
        eval_X, eval_y = train_dataset[index.item()]
        eval_X, eval_y = eval_X.to(device).unsqueeze(0), eval_y.to(device).unsqueeze(0)
        pred = model(eval_X, eval_y)
        indexes = torch.argmax(pred, dim=1)

        ru_tokens = list(map(lambda x: vocab_ru.get_itos()[x], eval_X[0].cpu().detach().numpy().tolist()))
        en_tokens = list(map(lambda x: vocab_eng.get_itos()[x], indexes.cpu().detach().numpy().tolist()))
        ru_tokens = ' '.join(list(filter(lambda x: x not in specials, ru_tokens
                                         )))
        en_tokens = ' '.join(list(filter(lambda x: x not in specials,en_tokens)))

        print(f'Epoch {i+1}, Train loss = {train_loss:.4}, test loss = {test_loss:.4}')
        print(f'Train Accuracy: {train_accuracy.compute():.2}, Test Accuracy: {test_accuracy.compute():.2}')

        print(f'RU: {ru_tokens}')
        print(f'EN: {en_tokens}')
        print('_'*50 + '\n')


Epoch 1, Train loss = 142.4, test loss = 35.68
Train Accuracy: 0.046, Test Accuracy: 0.044
RU: какие республики входили в ссср
EN: what the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
__________________________________________________

Epoch 26, Train loss = 95.17, test loss = 26.8
Train Accuracy: 0.083, Test Accuracy: 0.083
RU: кого в англии называют синими воротничками
EN: what is the the of
__________________________________________________

Epoch 51, Train loss = 88.31, test loss = 26.02
Train Accuracy: 0.095, Test Accuracy: 0.093
RU: как звали коня александра македонского
EN: what is the the of the of the
__________________________________________________

Epoch 76, Train loss = 83.81, test loss = 25.52
Train Accuracy: 0.096, Test Accuracy: 0.097
RU: в какой стране одно время правили сандинисты
EN: what what year is the the the
__________________________________________________

Epoch 101, Train loss = 77.21, test

In [74]:
res = []
true = []
model.eval()
for current_X, current_y in test_dataloader:
        current_X = current_X.to(device)
        current_y = current_y.to(device)
        pred = model(current_X, current_y)
        current_y = current_y[:, 1:]
        indexes = torch.argmax(pred, dim=1)
        current_y = current_y.flatten().cpu().detach().numpy().tolist()
        en_tokens = list(map(lambda x: vocab_eng.get_itos()[x], indexes.cpu().detach().numpy().tolist()))
        en_tokens = list(filter(lambda x: x not in set(['<eos>', '<sos>', '<pad>']), en_tokens))
        res.append(en_tokens)
        true_tokens = list(map(lambda x: vocab_eng.get_itos()[x], current_y))
        true_tokens = list(filter(lambda x: x not in set(['<eos>', '<sos>', '<pad>']), true_tokens))
        true.append([true_tokens])


In [75]:
from torchtext.data.metrics import bleu_score
bleu_score(res, true)

0.2000749111175537

## Обратная связь
- [ ] Хочу получить обратную связь по решению